In [17]:
import os
from os import walk
from os.path import join, split, splitext
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import argparse
import json
import numpy as np
import pandas as pd

# backend
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Conv1D, MaxPooling1D, concatenate, Concatenate, BatchNormalization, Dropout
from tensorflow.keras.models import Model, Sequential, model_from_json

from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
from sklearn.utils import class_weight

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
from tqdm import trange

from pathlib import Path

In [2]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

# tensorflow RNG
tf.random.set_seed(seed)

In [3]:
nClasses = 6
batch_size = 64
epoch = 20

MAX_FEATURES = 70000  # Size of vocabulary
EMBEDDING_DIM = MAX_FEATURES  # Size of vocabulary
SEQUENCE_LEN = 500 # Size of input arrays
EMBEDDING_OUT = 100  # Output dim of embedding

In [4]:
output_path = Path("./models")

In [5]:
data_path = Path("/mnt/nas/backups/08-07-2020/desktopg01/lisa/Data/CSV")

In [6]:
train = pd.read_csv(data_path/"train_small.csv", usecols=['document_type', 'body'])
val = pd.read_csv(data_path/"validation_small.csv", usecols=['document_type', 'body'])
test_data = pd.read_csv(data_path/"test_small.csv", usecols=['document_type', 'body'])

In [13]:
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train['body'])
with open(join(output_path, 'tokenizer.pickle'), 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
sequences_train = tokenizer.texts_to_sequences(train['body'])
sequences_validation = tokenizer.texts_to_sequences(val['body'])
sequences_test = tokenizer.texts_to_sequences(test_data['body'])

In [14]:
X_train = sequence.pad_sequences(sequences_train, maxlen=SEQUENCE_LEN, padding='post')
X_val = sequence.pad_sequences(sequences_validation, maxlen=SEQUENCE_LEN, padding='post')
X_test = sequence.pad_sequences(sequences_test, maxlen=SEQUENCE_LEN, padding='post')

In [15]:
encoder = LabelEncoder()
 
label = train['document_type']
label = encoder.fit_transform(label)

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(label),
                                                 label)
label = np.transpose(label)
label = to_categorical(label)

val_label = val['document_type'] 
val_label_toTest = encoder.transform(val_label)
val_label = np.transpose(val_label_toTest)
val_label = to_categorical(val_label)

test_label = test_data['document_type']
test_label_toTest = encoder.transform(test_label)
test_label = np.transpose(test_label_toTest)
test_label = to_categorical(test_label)

In [16]:
encoder.classes_, class_weights

(array(['acordao_de_2_instancia', 'agravo_em_recurso_extraordinario',
        'despacho_de_admissibilidade', 'outros', 'peticao_do_RE',
        'sentenca'], dtype=object),
 array([44.97197107,  9.76806756, 71.87716763,  0.18540788,  2.61536439,
        11.68130578]))

In [18]:
class F1History(tf.keras.callbacks.Callback):

    def __init__(self, train, validation=None):
        super(F1History, self).__init__()
        self.validation = validation
        self.train = train

    def on_epoch_end(self, epoch, logs={}):

        logs['F1_score_train'] = float('-inf')
        X_train, y_train = self.train[0], self.train[1]
        y_pred = (self.model.predict(X_train).ravel()>0.5)+0
        score = f1_score(y_train, y_pred, average="macro")       

        if (self.validation):
            logs['F1_score_val'] = float('-inf')
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_val_pred = (self.model.predict(X_valid).ravel()>0.5)+0
            val_score = f1_score(y_valid, y_val_pred, average="macro")
            logs['F1_score_train'] = np.round(score, 5)
            logs['F1_score_val'] = np.round(val_score, 5)
        else:
            logs['F1_score_train'] = np.round(score, 5)

In [19]:
f1_base = Input(shape=(SEQUENCE_LEN, ), dtype='int32')
text_embedding = Embedding(input_dim=MAX_FEATURES, output_dim=EMBEDDING_OUT,
                           input_length=SEQUENCE_LEN)(f1_base)

filter_sizes = [3, 4, 5]
convs = []
for filter_size in filter_sizes:
    l_conv = Conv1D(filters=256, kernel_size=filter_size, padding='same', activation='relu')(text_embedding)
    l_batch = BatchNormalization()(l_conv)
    l_pool = MaxPooling1D(2)(l_conv)
    
    convs.append(l_pool)

l_merge = Concatenate(axis=1)(convs)
l_pool1 = MaxPooling1D(50)(l_merge)
l_flat = Flatten()(l_pool1)
l_dense = Dense(128, activation='relu')(l_flat)
x = Dropout(0.5)(l_dense)
x = Dense(nClasses, activation='softmax')(x)
model = Model(inputs=f1_base, outputs=x)

In [20]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath=output_path/"stf_weights.keras", verbose=1, save_weights_only=True,
                               save_best_only=True, monitor="F1_score_val", mode="max")

model.fit(
	x=(X_train), y=(label),
	batch_size=batch_size,
	epochs=epoch,
	validation_data=(X_val, val_label),
	callbacks=[checkpointer,
               F1History(train=(X_train,label),validation=(X_val,val_label))],
	class_weight={i:v for i, v in enumerate(class_weights)})

Epoch 1/20
1355/2332 [================>.............] - ETA: 1:30 - loss: 1.1183 - accuracy: 0.5635

In [ ]:
# Convert Model into JSON Format
model_json = model.to_json()

with open(output_path/"cnn_text.json", "w") as json_file:
   json_file.write(model_json)

In [ ]:
model.load_weights(output_path/"stf_weights.keras")

In [ ]:
test_predict_1 = model.predict(X_val, verbose=1)
pred_1 = test_predict_1.argmax(axis=1)

target_names = ['acordao_de_2_instancia','agravo_em_recurso_extraordinario', 'despacho_de_admissibilidade', 'outros', 'peticao_do_RE', 'sentenca']
print(classification_report(val_label_toTest, pred_1, target_names=target_names, digits=4))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath=output_path/"stf_no_weights.keras", verbose=1, save_weights_only=True,
                               save_best_only=True, monitor="F1_score_val", mode="max")

model.fit(
	x=(X_train), y=(label),
	batch_size=batch_size,
	epochs=epoch,
	validation_data=(X_val, val_label),
	callbacks=[checkpointer,
               F1History(train=(X_train,label),validation=(X_val,val_label))])

In [ ]:
model.load_weights(output_path/"stf_no_weights.keras")

In [ ]:
test_predict_1 = model.predict(X_val, verbose=1)
pred_1 = test_predict_1.argmax(axis=1)

target_names = ['acordao_de_2_instancia','agravo_em_recurso_extraordinario', 'despacho_de_admissibilidade', 'outros', 'peticao_do_RE', 'sentenca']
print(classification_report(val_label_toTest, pred_1, target_names=target_names, digits=4))

In [ ]:
test_predict_1 = model.predict(X_test, verbose=1)
pred_1 = test_predict_1.argmax(axis=1)
print(classification_report(test_label_toTest,
                            pred_1, target_names=target_names, digits=4))